In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from DAFD.rv_study.rv_utils import *
from scipy.spatial import ConvexHull, convex_hull_plot_2d
from tqdm import tqdm

Using TensorFlow backend.


regime classifier
Loading classifier
Train accuracy: 0.9887387387387387

Regression model generation_rate2
Loading Regressor
R square (R^2) for Train:                 0.987384

Regression model droplet_size2
Loading Regressor
R square (R^2) for Train:                 0.983142

Regression model generation_rate1
Loading Regressor
R square (R^2) for Train:                 0.965872

Regression model droplet_size1
Loading Regressor
R square (R^2) for Train:                 0.953238



In [2]:
orthogonal_devices = pd.read_csv("DAFD/rv_study/orthogonal_devices.csv")

Run versatility analysis on 25 orthogonal devices and plot design space coverage

In [ ]:
devices = orthogonal_devices.to_dict(orient='records')
score = []
size_score = []
rate_score = []
all_points = {}
hulls = []

plt.figure(figsize=[8,5.5])
for device in tqdm(devices):
    features = device.copy()
    chip_num = features["Chip #"]
    del features["Chip #"]
    sizes, rates, _ = sweep_results(device, sweep_size=25, jet_drop=False, ca_range=[.05, .4])
    size_score.append(np.max(sizes) - np.min(sizes))
    rate_score.append(np.max(rates) - np.min(rates))
    points = np.array([[sizes[i], rates[i]] for i in range(len(sizes))])
    all_points[chip_num] = points
    hull = ConvexHull(points)
    hulls.append(hull)
    plt.scatter(points[:,0], points[:,1], s=10)
    for simplex in hull.simplices:
        plt.plot(points[simplex, 0], points[simplex, 1], 'k-')
    score.append(hull.volume)
orthogonal_devices["hull_object"] = hulls
plt.xticks([0, 100, 200, 300])
plt.xlabel("Droplet Size (um)")
plt.ylabel("Generation Rate (Hz)")
plt.xlim([0, 350])
plt.ylim([0, 500])
plt.title("Performance Span of Orthogonal Devices")


Calculate minimal orthogonal devices needed to cover design space

In [ ]:
# Making minimal setup
target = 0.99
coverage = 0.0
hulls = orthogonal_devices["hull_object"].to_dict()
devices_left = len(hulls)
points_left = []
points_used = []
used = []
areas = list(np.zeros(len(hulls)))

for key in all_points.keys():
    points_left.extend(all_points[key])
area_left = ConvexHull(points_left).volume
design_space_area = area_left

count = 1
while coverage <= target or count < 25:
    print(count)
    ## 1.1 Calculate area of each device hull
    for key in hulls:
        if key not in used:
            areas[key] = hulls[key].volume
    ## 1.2 pick hull with maximum area
    max_chip = areas.index(max(areas)) #requires 2 iterations but list is small
    used.append(max_chip)
    ## 1.3 remove all points WITHIN hull (& add to min dataset). This is for the full devices and for any overlaps
    points_left = []
    for chip_num in all_points.keys():
        points_placeholder = []
        for i, point in enumerate(all_points[chip_num]):
            if not in_hull(point, hulls[max_chip].points):
                points_placeholder.append(point)
            else:
                points_used.append(point)
        all_points[int(chip_num)] = np.array(points_placeholder) # Delete point if it is inside of the current point
        points_left.extend(all_points[chip_num])
        if chip_num-1 in used or len(all_points[chip_num]) < 3:
            areas[int(chip_num)-1] = -1
            if chip_num - 1 not in used:
                used.append(chip_num-1)
        else:
            hulls[int(chip_num)-1] = ConvexHull(all_points[chip_num])
        
    area_left = ConvexHull(points_used).volume
    coverage = area_left/design_space_area
    print("Iteration %f, Coverage %f" % (count, coverage))
    count+=1
    if count > 25:
        break
    if coverage >= 0.99:
        break

Plot minimal devices to visualize overlap and design space coverage

In [ ]:
entire_space = []
for key in all_points.keys():
    entire_space.extend(all_points[key])
entire_space= np.array(entire_space)
design_space_hull = ConvexHull(entire_space)

plt.figure(figsize=[8,5.5])
plt.scatter(all_points[11][:,0], all_points[11][:,1])
plt.scatter(all_points[21][:,0], all_points[21][:,1])
plt.scatter(all_points[1][:,0], all_points[1][:,1])
plt.scatter(all_points[22][:,0], all_points[22][:,1])
plt.scatter(all_points[3][:,0], all_points[3][:,1])

plt.scatter(all_points[25][:,0], all_points[25][:,1])
plt.scatter(all_points[24][:,0], all_points[24][:,1])

plt.legend(["11", "21", "1", "25", "24"])
plt.title("Minimal Design Space Coverage of Orthogonal Devices")
plt.xticks([0, 100, 200, 300])
plt.xlabel("Droplet Size (um)")
plt.ylabel("Generation Rate (Hz)")
for simplex in design_space_hull.simplices:
    plt.plot(entire_space[simplex, 0], entire_space[simplex, 1], 'k-')

plt.xlim([0, 350])
plt.ylim([0, 500])
#plt.savefig("minimal_plots.pdf")